In [1]:
import pandas as pd

In [2]:
exportdate = 20180327
projectname = 'repract'

In [3]:
df = pd.read_csv('../../data/papersummaries_cleaned.csv')
df.head()

,PaperID,PaperSummary
0,1,A method for automatically recovering software...
1,2,A set of two techniques for improving the qual...
2,3,A case study on evaluating a given technique f...
3,4,An experience report on the development of a m...
4,5,A document-driven study on the relevancy of cl...


In [4]:
longform_tags = pd.read_csv('../../analysis/papermapping/papertags_how_withwhom.csv')
longform_tags.head()

,PaperID,Tag,level_1,level_2,level_3
0,1,how:engineering:method,how,engineering,method
1,2,how:engineering:method,how,engineering,method
2,3,how:science:observation,how,science,observation
3,4,how:perspective:experience,how,perspective,experience
4,5,how:science:observation,how,science,observation


In [5]:
evaluation_counts = pd.read_csv(f'../../analysis/{exportdate}{projectname}_evaluation_counts.csv')
smaller = evaluation_counts[
    list(evaluation_counts.columns.values[:5])+['TotalRatings']].rename({'PaperId':'PaperID'}, axis=1)

In [6]:
smaller.head(2)

,PaperID,Essential,Worthwhile,Unimportant,Unwise,TotalRatings
0,248,2,5,6,0,13
1,100,7,5,0,0,12


In [7]:
def level_1_wrangling(longformdf, evaldf, level_1_tag):
    result = longformdf[longformdf.level_1 == level_1_tag].merge(evaldf, on='PaperID'
                                                                ).groupby(['level_1', 'level_2', 'level_3',
                            'Essential', 'Worthwhile','Unimportant','Unwise', 'TotalRatings']).count(
    ).reset_index().groupby(['level_1', 'level_2', 'level_3']).sum(
    ).sort_values(['Essential','Worthwhile'],ascending=False).T[:-2].T
    for col in result.columns.values: 
        if col != 'TotalRatings':
            result[col] = result[col] / result['TotalRatings']
    result['PosPercentage'] = result['Essential'] + result['Worthwhile']
    return result.sort_values(['PosPercentage', 'Essential'], ascending=False)

In [8]:
level_1_wrangling(longform_tags, smaller, 'how')

Essential  Worthwhile  Unimportant  \
level_1 level_2     level_3                                             
how     science     observation     0.251515    0.475758     0.230303   
                    interrogation   0.315789    0.410526     0.236842   
        engineering method          0.303393    0.419162     0.215569   
                    analysis        0.200000    0.500000     0.200000   
                    technology      0.222222    0.462963     0.277778   
        perspective opinion         0.177778    0.488889     0.311111   
                    philosophy      0.166667    0.500000     0.166667   
                    experience      0.160494    0.487654     0.314815   
                    review          0.176471    0.441176     0.338235   
        science     intervention    0.112500    0.475000     0.343750   

                                     Unwise  TotalRatings  PosPercentage  
level_1 level_2     level_3                                               
how     science     observation    0.042424           330       0.727273  
                    interrogation  0.036842           190       0.726316  
        engineering method         0.061876           501       0.722555  
                    analysis       0.100000            10       0.700000  
                    technology     0.037037           216       0.685185  
        perspective opinion        0.022222            45       0.666667  
                    philosophy     0.166667             6       0.666667  
                    experience     0.037037           162       0.648148  
                    review         0.044118            68       0.617647  
        science     intervention   0.068750           160       0.587500

In [9]:
level_1_wrangling(longform_tags, smaller, 'withwhom')

Essential  Worthwhile  Unimportant  \
level_1  level_2       level_3                                             
withwhom professionals practitioners   0.319149    0.460993     0.205674   
         laypeople     students        0.090226    0.488722     0.353383   
         professionals academics       0.100000    0.400000     0.200000   
         laypeople     others          0.000000    0.000000     1.000000   

                                        Unwise  TotalRatings  PosPercentage  
level_1  level_2       level_3                                               
withwhom professionals practitioners  0.014184           141       0.780142  
         laypeople     students       0.067669           133       0.578947  
         professionals academics      0.300000            10       0.500000  
         laypeople     others         0.000000             1       0.000000

# TBC